# Function definition tracing

This notebook is a MVP of function definition tracing.

## Requirements

```shell
pip install rope, asttokens
pip install tests/resources/tliba
```

In [ ]:
import ast
from ast import NodeVisitor
import inspect
import importlib
from pathlib import Path

import asttokens
from rope.base.project import Project
from rope.base.libutils import path_to_resource, analyze_modules
from rope.contrib.findit import find_definition, Location

In [ ]:
from rope.contrib.findit import occurrences

In [ ]:
class CallVisitor(NodeVisitor):
    def __init__(self, project, resource, code, tree_tokens):
        self.project = project
        self.resource = resource
        self.code = code
        self.tree_tokens = tree_tokens
        self.calls = []

    def visit_Call(self, node: ast.Call):
        loc = find_definition(
            self.project,
            self.code,
            self.tree_tokens.get_text_range(node)[0]
        )
        if loc is not None:
            if loc.resource is None:
                loc.resource = self.resource
            self.calls.append((node, loc))
        super().generic_visit(node)


In [ ]:
class ModuleStore:
    pkg: str
    name: str
    path: str
    code: str
    tree: ast.Module
    tree_tokens: asttokens.ASTTokens
    

In [ ]:
def get_project(pkg: str, projects: dict[str, Project]) -> Project:
    if pkg in projects:
        return projects[pkg]
    # Once for each package that we want to cover
    projects[pkg] = Project(importlib.util.find_spec(pkg).submodule_search_locations[0])
    proj = projects[pkg]
    analyze_modules(proj)
    return proj

In [ ]:
def get_call_definitions(func, modules, projects):
    mod = inspect.getmodule(func)
    if mod.__name__ not in modules:
        pkg, _sep, _stem = mod.__name__.partition('.')
        ms = ModuleStore()
        ms.name = mod.__name__
        ms.pkg = pkg
        ms.path = Path(inspect.getsourcefile(mod))
        ms.code = ms.path.read_text()
        ms.tree = ast.parse(ms.code)
        ms.tree_tokens = asttokens.ASTTokens(ms.code, parse=False, tree=ms.tree)
        modules[ms.name] = ms
    else:
        ms = modules[mod.__name__]

    proj = get_project(ms.pkg, projects)    
    resource = path_to_resource(proj, ms.path)

    visitor = CallVisitor(proj, resource, ms.code, ms.tree_tokens)
    visitor.visit(ms.tree)
    return visitor.calls

In [ ]:
projects = {}
modules = {}

In [ ]:
from tliba import compute_moments

In [ ]:
calls = get_call_definitions(compute_moments, modules, projects)

In [ ]:
node, location = calls[0]

In [ ]:
from ast import NodeTransformer

In [ ]:
import hashlib

In [ ]:
class ReplaceCall(NodeTransformer):
    def visit_Call(self, node: ast.Call):
        self.hash_repr = hashlib.sha256(ast.unparse(node).encode()).hexdigest()
        super().generic_visit(node)
        return node

    def visit_Name(self, node: ast.Name):
        if self.hash_repr is not None:
            node.id = self.hash_repr
            self.hash_repr = None
        return node


In [ ]:
ast.unparse(node)

In [ ]:
ast.unparse(ReplaceCall().visit(node))

In [ ]:
ast.dump(node)

In [ ]:
node.func.value.func.id

In [ ]:
ast.dump(node)

In [ ]:
node.first_token

In [ ]:
location

In [ ]:
location.region[0]

In [ ]:
from rope.refactor import occurrences

In [ ]:
from types import FunctionType

In [ ]:
import rope

In [ ]:
def get_func_def_location(func: FunctionType, project_store):
    func_name = func.__name__
    module_name = func.__module__
    for project in project_store.get_projects():
        module = project.get_module(module_name)
        if isinstance(module, rope.base.pyobjectsdef.PyModule):
            break

    finder = occurrences.Finder(project, func_name)
    for occurrence in finder.find_occurrences(pymodule=module):
        return Location(occurrence)

In [ ]:
from pycodehash.tracing.stores import ProjectStore

In [ ]:
project_store = ProjectStore()

In [ ]:
project_store.set("tliba")

In [ ]:
get_func_def_location(compute_moments, project_store)

In [ ]:
module = modules['tliba.summary']

In [ ]:
project = projects["tliba"]

In [ ]:
node_token_range = module.tree_tokens.get_text_range(node)
location = find_definition(
    projects["tliba"],
    module.code,
    offset=node_token_range[0]
)

In [ ]:
location

In [ ]:
def hash_func(location, project):
    module = project.get_pymodule(location.resource)
    fname = location.resource.read()[location.region[0]:location.region[1]]
    func = rope_mod.get_attribute(fname).get_object()
    node = func.ast_node

In [ ]:
module = project.get_pymodule(location.resource)
fname = location.resource.read()[location.region[0]:location.region[1]]
func = module.get_attribute(fname).get_object()
node = func.ast_node

In [ ]:
node

In [ ]:
func.get_ast()

In [ ]:
func

In [ ]:
location.resource.read()[location.region[0]:location.region[1]]

In [ ]:
import rope

In [ ]:
from pycodehash.tracing.stores import ModuleView

In [ ]:
module.get_resource().pathlib

In [ ]:
module.resource.read()

In [ ]:
def _create_module_view(mod: rope.base.pyobjectsdef.PyModule) -> ModuleView:
    name = mod.get_name()
    pkg, _sep, _stem = name.partition(".")
    path = module.get_resource().pathlib
    code = module.resource.read()
    tree = module.get_ast()

    return ModuleView(
        pkg=pkg,
        name=name,
        path=path,
        code=code,
        tree=tree,
        tree_tokens=asttokens.ASTTokens(code, parse=False, tree=tree),
    )


In [ ]:
_create_module_view(module)

In [ ]:
type(module)

In [ ]:
module.get_name()

In [ ]:
func.get_ast()

In [ ]:
module

In [ ]:
print(location.resource.read())

In [ ]:
modules["tliba.summary"].code[145:208]

In [ ]:
modules["tliba.summary"].tree_tokens.get_text_range(node)

In [ ]:
node.lineno

In [ ]:
ast.unparse(node)

In [ ]:
projects["tliba"].find_module("summary")